# シュレッダー復元問題の定式化

## 二次割当て問題としての定式化

### 定数とインデックス

- $N$: 紙片の総数
- $i,j \in [1,\dots,N]$: 位置を表すインデックス
- $a,b \in [1,\dots,N]$: 紙片を表すインデックス
- $d \in \{r, c\}$: 連結の方向
- $S_{abd}$: 紙片 $a$ と $b$ が方向 $d$ で連結した場合の類似度
- $D_{ijd}$: 位置 $i$ と $j$ が方向 $d$ でつながっているか否かのフラグ

### 変数

- $x_{ia}$: 位置 $i$ に紙片 $a$ が置かれている場合は $1$ となるフラグ

### 目的関数

$$
\max \sum_{abijd}S_{abd}D_{ijd}x_{ia}x_{jb}
$$

### 制約条件

- $\sum_{i}x_{ia} = 1$: 紙片 $a$ は、どこか一箇所にしか置かれない
- $\sum_{a}x_{ia} = 1$: 位置 $i$ における紙片は、1つである


## MIP への変形

$x_{ia}x_{jb}$を表す補助変数 $y_{ijab}$ を導入する

|$x_{ai}$|$x_{jb}$|$y_{ijab}$|
|:--:|:--:|:--:|
|0|0|0|
|0|1|0|
|1|0|0|
|1|1|1|

これは以下の制約で実現できる

- $x_{ia} + x_{jb} \leq y_{ijab} + 1$
- $x_{ia} \geq y_{ijab}$
- $x_{jb} \geq y_{ijab}$


In [143]:
import numpy as np
from functools import lru_cache
from collections import defaultdict
import pathlib
import pickle


def get_sim_func(imgs):
    @lru_cache(maxsize=None)
    def sim(a, b, d):
        height = min(imgs[a].shape[0], imgs[b].shape[0])
        width = min(imgs[a].shape[1], imgs[b].shape[1])
        if d == 'r':
            return imgs[a][-1, :, :][:height]
            rhs = imgs[b][0, :, :][:height]
        elif d == 'c':
            lhs = imgs[a][:, -1, :][:, :width]
            rhs = imgs[b][:, 0, :][:, :width]
        else:
            raise
        return (lhs == rhs).sum()
    return sim
        

def sim(a, b, d):
    if d == 'r':
        lhs = a[-1, :, :][:b.shape[0]]
        rhs = b[0, :, :][:a.shape[0]]
    elif d == 'c':
        lhs = a[:, -1, :][:, :b.shape[1]]
        rhs = b[:, 0, :][:, :a.shape[1]]
    else:
        raise
    return (lhs == rhs).sum()


def total_cost(imgs, rows, cols):
    cost = 0
    n = rows*cols
    for i in range(len(imgs)):
        if (1 + 1)%cols > 0:
            cost += sim(imgs[i], imgs[i + 1], 'r')
        if i + cols < n:
            cost += sim(imgs[i], imgs[i + cols], 'c')
    return cost


def get_coeff(imgs, rows, cols):
    """
    \sum_{d}S_{abd}D_{ijd} を返す
    """
    n = rows*cols
    coeff = defaultdict(float)
    for a in range(len(imgs)):
        for b in range(len(imgs)):
            for i in range(rows*cols):
                if (i + 1)%cols > 0:
                    coeff[a, b, i, i + 1] = sim(imgs[a], imgs[b], 'r')
                if i + cols < n:
                    coeff[a, b, i, i + cols] = sim(imgs[a], imgs[b], 'c')
    return coeff

In [144]:
IMG_DIR = '../data/shred'

imgs = sorted(pathlib.Path(IMG_DIR).glob('*.pkl'))

In [145]:
with open(imgs[12], 'rb') as i_:
    i = pickle.load(i_)

In [146]:
rows = max(list(zip(*i[1]))[0]) + 1
cols = max(list(zip(*i[1]))[1]) + 1
n = rows*cols

In [147]:
coeff = get_coeff(i[0], rows, cols)

In [126]:
import pulp


problem = pulp.LpProblem(sense=pulp.LpMaximize)

x = np.array(pulp.LpVariable.matrix('x', (range(n), range(n)), cat=pulp.LpBinary))
y = np.array(pulp.LpVariable.matrix('y', (range(n), range(n), range(n), range(n)), cat=pulp.LpBinary))

problem.setObjective(pulp.lpSum(
    coeff[a, b, i, j]*y[a, b, i, j] if coeff[a, b, i, j] != 0 else 0
    for a in range(n)
    for b in range(n)
    for i in range(n)
    for j in range(n)
    if i != j
))

for i in range(n):
    problem.addConstraint(pulp.lpSum(x[:, i]) == 1)

for a in range(n):
    problem.addConstraint(pulp.lpSum(x[a, :]) == 1)

for a, b, i, j in coeff.keys():
    if coeff[a, b, i, j] != 0:
        problem.addConstraint(x[a, i] + x[b, j] <= y[a, b, i, j] + 1)
        problem.addConstraint(x[a, i] >= y[a, b, i, j])
        problem.addConstraint(x[b, j] >= y[a, b, i, j])

In [127]:
problem.solve()

1

In [119]:
total_cost()

pkl'), PosixPath('../data/shred/chabudai_kaeshi_50_2.pkl'), PosixPath('../data/shred/chabudai_kaeshi_50_4.pkl'), PosixPath('../data/shred/chabudai_kaeshi_5_1.pkl'), PosixPath('../data/shred/chabudai_kaeshi_5_2.pkl'), PosixPath('../data/shred/chabudai_kaeshi_5_4.pkl'), PosixPath('../data/shred/depositphotos_11056146-stock-photo-two-peeking-from-hole-in_100_1.pkl'), PosixPath('../data/shred/depositphotos_11056146-stock-photo-two-peeking-from-hole-in_100_2.pkl'), PosixPath('../data/shred/depositphotos_11056146-stock-photo-two-peeking-from-hole-in_100_4.pkl'), PosixPath('../data/shred/depositphotos_11056146-stock-photo-two-peeking-from-hole-in_10_1.pkl'), PosixPath('../data/shred/depositphotos_11056146-stock-photo-two-peeking-from-hole-in_10_2.pkl'), PosixPath('../data/shred/depositphotos_11056146-stock-photo-two-peeking-from-hole-in_10_4.pkl'), PosixPath('../data/shred/depositphotos_11056146-stock-photo-two-peeking-from-hole-in_30_1.pkl'), PosixPath('../data/shred/depositphotos_11056146-s

KeyboardInterrupt: 